In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import pycaret
from pycaret.regression import *
from pycaret.classification import *

In [14]:
data = pd.read_csv('Trang_clean.csv')
data.drop(['Date', 'Time'], axis=1, inplace=True)
data

,PM25,PM10,O3,CO,NO2,SO2,WS,TEMP,RH,WD
0,11.5,20.0,23.0,0.20,1.0,0.0,2,28,86,15
1,11.2,18.0,22.0,0.27,2.0,0.0,2,27,88,352
2,9.8,17.0,19.0,0.26,3.0,0.0,2,27,90,348
3,9.7,17.0,17.0,0.26,2.0,0.0,2,26,91,1
4,8.7,16.0,17.0,0.26,2.0,0.0,2,27,90,3
...,...,...,...,...,...,...,...,...,...,...
1424,20.4,36.0,3.0,0.13,5.0,0.0,1,24,92,283
1425,15.2,32.0,1.0,0.17,6.0,0.0,0,24,94,275
1426,13.7,33.0,0.0,0.20,7.0,0.0,1,24,93,319
1427,16.8,40.0,3.0,0.21,8.0,0.0,1,26,88,343


In [18]:
session_1 = setup(data, target='PM25', train_size=0.8)

ValueError: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.